In [ ]:
R.home()

In [ ]:
baizer::pkglib(tidyverse, Biostrings, Peptides, baizer, genogamesh, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])
ab_scheme <- str_to_lower(config[['ext_numbering']])

In [ ]:
# read from igblast
TBigblast_tsv <- read_tsv(snakemake@input[['VDJB_igblast_tsv']])
TBshm <- parse_IgBlast_shm(snakemake@input[['VDJB_igblast_txt']])

# read from cellranger
TBcr_airr <- read_tsv(str_c(snakemake@input[['count_dir']], str_glue(config[['count_VDJB_airr']]), sep='/'))
TBcr_anno <- read_csv(str_c(snakemake@input[['count_dir']], str_glue(config[['count_VDJB_anno']]), sep='/'))

# read from changeo
TBchangeo <- bind_rows(read_tsv(snakemake@input[['VDJB_changeo']]), read_tsv(snakemake@input[['VDJB_changeo_fail']]))

# read from anarci
TBext_anarci_H <- read_csv(snakemake@input[['VDJB_ext_anarci_H']])
TBext_anarci_L <- read_csv(snakemake@input[['VDJB_ext_anarci_KL']])
TBext_anarci_seq <- bind_rows(
    parse_ANARCI_aaseq(TBext_anarci_H, chain='H', scheme=ab_scheme),
    parse_ANARCI_aaseq(TBext_anarci_L, chain='L', scheme=ab_scheme)
) %>% rename_at(str_subset(colnames(.), 'cdr\\d_aa'), ~str_c(ab_scheme, .x, sep='_'))

In [ ]:
# main and fine x_call
gene_cols <- str_subset(colnames(TBigblast_tsv), '_call$')

TBigblast_tsv <- TBigblast_tsv %>%
    # extract first fine gene to _fine cols
    mutate(across(all_of(gene_cols), ~reg_match(.x, '[^,]+'), .names='{.col}_fine')) %>%
    # extract first main gene 
    mutate(across(all_of(gene_cols), ~reg_match(.x, '[^\\*]+'))) %>%
    # rename
    rename_at(c(gene_cols, str_c(gene_cols, '_fine')), 
              ~str_replace_all(.x, '_call', '_gene'))

# cdr length, the length of NA cdr will be NA
cdr_aa_cols <- str_subset(colnames(TBigblast_tsv), 'cdr\\d_aa')
TBigblast_tsv <- TBigblast_tsv %>%
    mutate(across(all_of(cdr_aa_cols), ~nchar(.x), .names='{.col}_len'))

# rename shm columns

TBshm <- TBshm %>% rename_at(-1, ~str_to_lower(.x) %>% str_c('_mismatch_nt')) %>%
    mutate(v_domain_shm_nt = 
           sum(c(v_mismatch_nt, d_mismatch_nt, j_mismatch_nt), na.rm=TRUE), 
           .by=sequence_id)

# cdr3 aa feature, the feature of NA cdr3 will be NA
TBaafeature <- TBigblast_tsv %>% select(sequence_id, cdr3_aa) %>% mutate(
    cdr3_charge=ifelse(!is.na(cdr3_aa), charge(seq=cdr3_aa, pH=7.4, pKscale='EMBOSS'), NA),
    cdr3_hydrophobicity=map_dbl(cdr3_aa, ~ifelse(!is.na(.x), hydrophobicity(seq=.x, scale='Eisenberg'), NA)),
    cdr3_boman=map_dbl(cdr3_aa, ~ifelse(!is.na(.x), boman(seq=.x), NA)),
    # if nchar of cdr3 <= 2, instaIndex will throw an error
    cdr3_instaindex = map_dbl(cdr3_aa, ~ifelse(!is.na(.x) & nchar(.x) > 2, instaIndex(seq=.x), NA))
) %>% select(-cdr3_aa)

In [ ]:
##################################
### select cols
##################################

In [ ]:
# columns from igblast
TBigblast_sel <- TBigblast_tsv %>% select(
                    sequence_id, productive_igblast=productive, chain=locus, 
                    v_gene, d_gene, j_gene, v_gene_fine, d_gene_fine, j_gene_fine, 
                    cdr1_aa_len, cdr2_aa_len, cdr3_aa_len,
                    cdr1_nt=cdr1, cdr1_aa, cdr2_nt=cdr2, cdr2_aa, cdr3_nt=cdr3, cdr3_aa, np1_nt=np1, np2_nt=np2,
                    fwr1_nt=fwr1, fwr1_aa, fwr2_nt=fwr2, fwr2_aa, fwr3_nt=fwr3, fwr3_aa, fwr4_nt=fwr4, fwr4_aa,
                    v_start=v_sequence_start, v_end=v_sequence_end, 
                    d_start=d_sequence_start, d_end=d_sequence_end,
                    j_start=j_sequence_start, j_end=j_sequence_end,
                    seq_nt=sequence, seq_align_nt=sequence_alignment, gm_align_nt=germline_alignment
        ) 

# columns from shm
TBshm_sel <- TBshm %>%
    relocate(sequence_id, v_domain_shm_nt, contains('cdr'), contains('fwr'))

# columns from cellranger annotation
TBcr_annot_sel <- TBcr_anno %>% select(cell=barcode, sequence_id=contig_id, 
                                       productive_cellranger=productive, clone_cellranger=raw_clonotype_id, reads, umis, c_gene)

# columns from cellranger airr
TBcr_airr_sel <- TBcr_airr %>% select(sequence_id, seq_aa=sequence_aa)

# columns from changeo
TBchangeo_sel <- TBchangeo %>% select(sequence_id, clone_changeo=clone_id, seq_gm=germline_alignment_d_mask)

# columns from anarci
TBext_anarci_seq <- TBext_anarci_seq

In [ ]:
##################################
### join
##################################

In [ ]:
TBjoin <- TBigblast_sel %>% left_join(TBshm_sel, by='sequence_id') %>%
    relocate(all_of(colnames(TBshm_sel)[-1]), .after=cdr3_aa_len)

TBjoin <- TBjoin %>% left_join(TBcr_annot_sel, by='sequence_id') %>%
    relocate(cell, productive_cellranger, .after=sequence_id) %>%
    relocate(clone_cellranger, reads, umis, c_gene, .after=chain)

TBjoin <- TBjoin %>% left_join(TBcr_airr_sel, by='sequence_id') %>%
    relocate(seq_aa, .after=seq_nt)

TBjoin <- TBjoin %>% left_join(TBchangeo_sel, by='sequence_id') %>%
     relocate(clone_changeo, .after=clone_cellranger)

TBjoin <- TBjoin %>% left_join(TBext_anarci_seq, by='sequence_id') %>%
    relocate(matches('cdr\\d_aa'), .after=fwr4_aa) %>%
    relocate(seq_align_aa, .after=seq_align_nt)

TBjoin <- TBjoin %>% left_join(TBaafeature, by='sequence_id') %>%
    relocate(all_of(colnames(TBshm_sel)[-1]), .after=cdr3_aa)

In [ ]:
# calculate shm ratio
TBjoin <- TBjoin %>% mutate(v_domain_shm_ratio = round(v_domain_shm_nt/nchar(seq_align_nt), 4), 
                  .after=v_domain_shm_nt)

In [ ]:
##################################
### widen for HL
##################################

In [ ]:
TBjoin <- TBjoin %>% mutate(HL=case_when(chain=='IGH' ~ 'H', chain=='IGL' ~ 'L', chain=='IGK' ~ 'L')) %>%
    mutate(HL=factor(HL, c('H', 'L'))) %>% 
    # remove the contigs unknown
    filter(!is.na(HL))

In [ ]:
# contig numbers of a cell, and whether there is only an unique H or L
TBunique <- TBjoin %>% group_by(cell, HL) %>% summarise(contig_num = n(), unique = n() == 1) %>%
    ungroup

In [ ]:
# for multi-contigs, only keep the first one with most umis
TBjoin <- TBjoin %>% group_by(cell, HL) %>% arrange(desc(umis)) %>% dplyr::slice(1) %>% ungroup %>%
    arrange(HL, cell, sequence_id)

In [ ]:
TBjoin <- TBjoin %>% left_join(TBunique, by=c('cell', 'HL')) %>%
    relocate(contig_num, unique, .after=umis)

In [ ]:
# pivot
keep_col <- c('cell', 'HL')
TBwider <- TBjoin %>% pivot_wider(names_from='HL', values_from=-all_of(keep_col))

In [ ]:
# unique, productive, clone_cellranger, class
TBwider <- TBwider %>% 
    mutate(productive_igblast=productive_igblast_H & productive_igblast_L,
           productive_cellranger=productive_cellranger_H & productive_cellranger_H,
           unique = unique_H & unique_L, 
           class=str_sub(c_gene_H, 1, 4) %>% str_replace('GH', 'g'),
           clone_cellranger = case_when(!is.na(clone_cellranger_H) ~ clone_cellranger_H,
                                        !is.na(clone_cellranger_L) ~ clone_cellranger_L),
           clone_changeo=str_glue('{clone_changeo_H}-{clone_changeo_L}'),
           .after=cell
          )
# if single contig
TBwider <- TBwider %>% mutate(
    productive_igblast=ifelse(is.na(seq_nt_H) | is.na(seq_nt_L), FALSE, productive_igblast),
    productive_cellranger=ifelse(is.na(seq_nt_H) | is.na(seq_nt_L), FALSE, productive_cellranger),
    unique=ifelse(is.na(seq_nt_H) | is.na(seq_nt_L), FALSE, unique)
)

In [ ]:
##################################
### mutation target
##################################

In [ ]:
# Hchain mut
orfH <- parse_ANARCI_aaseq(read_csv(snakemake@input[['VDJB_orf_anarci_H']]), chain='H', keep_number=TRUE) %>% 
    select(id=sequence_id, matches('^\\d')) %>% c2r('id')
gmH <- parse_ANARCI_aaseq(read_csv(snakemake@input[['VDJB_gm_anarci_H']]), chain='H', keep_number=TRUE) %>% 
    select(id=sequence_id, matches('^\\d')) %>% c2r('id')
common_ids <- intersect(rownames(orfH), rownames(gmH))
common_cols <- intersect(colnames(orfH), colnames(gmH))
diff_bool_H <- orfH[common_ids, common_cols] != gmH[common_ids, common_cols]
# if germline is -
diff_bool_H <- ifelse(gmH != '-', diff_bool_H, FALSE)
colnames(diff_bool_H) <- str_c('imgt_H', colnames(diff_bool_H))

In [ ]:
# Lchain mut
orfL <- parse_ANARCI_aaseq(read_csv(snakemake@input[['VDJB_orf_anarci_KL']]), chain='L', keep_number=TRUE) %>% 
    select(id=sequence_id, matches('^\\d')) %>% c2r('id')
gmL <- parse_ANARCI_aaseq(read_csv(snakemake@input[['VDJB_gm_anarci_KL']]), chain='L', keep_number=TRUE) %>% 
    select(id=sequence_id, matches('^\\d')) %>% c2r('id')
common_ids <- intersect(rownames(orfL), rownames(gmL))
common_cols <- intersect(colnames(orfL), colnames(gmL))
diff_bool_L <- orfL[common_ids, common_cols] != gmL[common_ids, common_cols]
# if germline is -
diff_bool_L <- ifelse(gmL != '-', diff_bool_L, FALSE)
colnames(diff_bool_L) <- str_c('imgt_L', colnames(diff_bool_L))

In [ ]:
TBmut <- TBwider %>% select(cell, sequence_id_H, sequence_id_L) %>%
    left_join(r2c(as.data.frame(diff_bool_H), 'sequence_id_H'), by='sequence_id_H') %>%
    left_join(r2c(as.data.frame(diff_bool_L), 'sequence_id_L'), by='sequence_id_L') %>%
    select(-matches('^sequence_id'))
TBmut <- TBmut %>% mutate(batch = sample, .after=cell)

In [ ]:
TBwider <- TBwider %>% left_join(TBmut, by='cell')
TBwider <- TBwider %>% mutate(batch = sample, .after=cell)

In [ ]:
# stat

TBmtx <- read_csv(str_c(snakemake@input[['count_dir']], str_glue(config[['count_VDJB_mtx']]), sep='/')) %>% 
    dplyr::rename(item=`Metric Name`, value=`Metric Value`) %>%
    filter(`Library Type` == 'VDJ B')

In [ ]:
Lstat <- TBmtx %>% distinct(item, value) %>% pivot_wider(names_from='item', values_from='value') %>% select(any_of(c(
    'cells'='Estimated number of cells', 
    'mean_reads'='Mean reads per cell', 
    'median_IGH'='Median IGH UMIs per Cell', 
    'median_IGK'='Median IGK UMIs per Cell', 
    'median_IGL'='Median IGL UMIs per Cell', 
    'align_rate'='Reads mapped to any V(D)J gene', 
    'total_reads'='Number of reads', 
    'Q30_barcode'='Q30 barcodes', 
    'Q30_umi'='Q30 UMI', 
    'Q30_R1'='Q30 RNA read',
    'Q30_R2'='Q30 RNA read 2'
))) %>% as.list

In [ ]:
# write
dir.create(dirname(snakemake@output[['VDJB_csv']]), recursive = TRUE)
write_excel_csv(TBmut, snakemake@output[['VDJB_mut']])
write_excel_csv(TBwider, snakemake@output[['VDJB_csv']])
write_yaml(Lstat, file=snakemake@output[['VDJB_stat']])

In [ ]:
dir.create(snakemake@output[['stat_dir']], recursive = TRUE)
file.copy(snakemake@output[['VDJB_csv']], 
          str_c(snakemake@output[['stat_dir']], '/', basename(snakemake@output[['VDJB_csv']])), 
          overwrite=TRUE)
file.copy(snakemake@output[['VDJB_stat']], 
          str_c(snakemake@output[['stat_dir']], '/', basename(snakemake@output[['VDJB_stat']])), 
          overwrite=TRUE)
# copy changeo
file.copy(snakemake@input[['VDJB_changeo']], 
          str_c(snakemake@output[['stat_dir']], '/', basename(snakemake@input[['VDJB_changeo']])), 
          overwrite=TRUE)